# Merge the files recovered from vox-grn into our database for processing.

The successful recovery of files from vox-grn now leaves the task of integrating the files with our existing data. The major issue is that the vox-grn data does not have any information at the item granularity. This is a problem mainly because the item type is important to filter out instrumentals and singing - although singing could be a sub-group that we export.

It has been noted that some information can be extracted from the filename. The file name appears to follow this pattern:

> ./Audio_MP3/[nn]/[ppppp]/[Language name][Track Title][ttt][Item 1 title] ♦ [Item 2 title] [...] [ppppp].mp3

where:

* nn are the first two digits of the program
* ppppp is the program number zero padded to always be 5 digits.
* ttt is the track number zero padded to always be 3 digits.

There are as many ♦ separated item titles as there are items in the track.

Our aim here is to preserve as much meta-data as we can and associate files with program items. We do not filter out based on item type - that can be done later.

In [1]:
import pandas as pd
import numpy as np
import os
import sys
from pathlib import Path
import requests
import json


/opt/conda/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


## Gather Data
First read in the descriptors of vox-grn and grid data.


In [2]:
# Now read in the description of the input and remove the unwanted columns and rename the rest to be python attribute names.
file_descriptors = pd.read_csv("/prometheus/GRN/recording_files_with_tags_and_track.csv")
print(f'File descriptors shape: {file_descriptors.shape}')
items = pd.read_csv("/prometheus/GRN/grid_program_items.csv")
print(f'Program items shape: {items.shape}')


File descriptors shape: (210704, 12)
Program items shape: (267681, 21)


In [3]:
# create a dataframe using a generator
def gen_vox_grn():
  resp = requests.get('https://raw.githubusercontent.com/johno-source/vox-grn/main/data/vox-grn.json')
  vox_dict = json.loads(resp.text)
  for iso in vox_dict.keys():
    lang_df = pd.json_normalize(vox_dict[iso])
    lang_df['iso'] = iso
    yield lang_df

vox_df = pd.concat(gen_vox_grn())

# Input Data Description
Give a description of the columns and size of the input data.

In [4]:
print(f'vox-grn columns: {vox_df.columns}')
print(f'recording_files_with_tags_and_track columns: {file_descriptors.columns}')
print(f'program items columns: {items.columns}')


vox-grn columns: Index(['file', 'language name', 'location', 'copyright', 'year', 'disguised',
       'length', 'iso'],
      dtype='object')
recording_files_with_tags_and_track columns: Index(['LanguageID', 'ISO', 'Language', 'Program', 'Track', 'Recordist',
       'Location', 'Year', 'Path', 'Filename', 'Size', 'Length'],
      dtype='object')
program items columns: Index(['Program Number', 'Program Item Number', 'Tape Side', 'Track Number',
       'Original Recording Number', 'Original Item Number', 'Title',
       'Vernacular Item Title', 'Language Number', 'Language Name',
       'Item Start Time', 'Item Time', 'Finish Time', 'Original Time',
       'Script Number', 'Script Name', 'Picture Number', 'Item Type',
       'Comments', 'Entered By', 'Enter On Date'],
      dtype='object')


# Common Identity

To tie all the items together we need to give them a common identity. The best thing to use is the program and the track. We use these to create an ID with the format [ppppp]_[ttt].

In [5]:
# pull out the program ID and the track from each data source
vox_df['program'] = vox_df['file'].str.extract('\./Audio_MP3/[0-9]{2}/([0-9]{5})')
vox_df['track'] = vox_df['file'].str.extract('\./Audio_MP3/[0-9]{2}/[0-9]{5}/.*?([0-9]{3})')
items['program'] = items['Program Number'].str[1:]
file_descriptors['prog'] = file_descriptors['Program'].str[1:]

In [6]:
vox_df['ID'] = vox_df['program'] + '_' + vox_df['track']
items['ID'] = items['program'] + '_' + items['Track Number'].astype(int).apply('{:0>3d}'.format)
file_descriptors['ID'] = file_descriptors['prog'] + '_' + file_descriptors['Track'].astype(int).apply('{:0>3d}'.format)

# Missing Files
Recreate the list of files that were downloaded.


In [7]:
def check_for_file(item_row):
    return os.path.isfile('/media/programs/' + item_row['Path'] + item_row['Filename'] )

file_descriptors['file exists'] = file_descriptors.apply(check_for_file, axis=1)
print(f'Files verified as existing for {sum(file_descriptors["file exists"])} out of {file_descriptors.shape[0]} records.')

Files verified as existing for 203879 out of 210704 records.


In [8]:
missing_files = file_descriptors[file_descriptors['file exists'] == False].copy()

In [9]:

missing_files['found_prog'] = missing_files['prog'].isin(vox_df['program'])
found_files = missing_files[missing_files['found_prog']].copy()
print(f'Found {len(found_files)} out of {len(missing_files)} missing files.')

Found 6604 out of 6825 missing files.


# Recovered Files

We actually downloaded any file that had a program id that was contained in the missing files.


In [14]:
vox_df['recovery_candidate'] = vox_df['program'].isin(missing_files['prog'])
recovered_files = vox_df[vox_df['recovery_candidate']].copy()
print(f'Recovery files: {len(recovered_files)}')

Recovery files: 7015


We need to put the path and file name into separate fields for merging.

In [15]:
def extract_vox_grn_key(vox_grn_dir):
    return vox_grn_dir[2:]

def extract_vox_path(vox_grn_dir):
    vox_path_list = extract_vox_grn_key(vox_grn_dir).split('/')
    vox_path = '.'
    if len(vox_path_list) > 1:
        vox_path = 'vox_grn/' + '/'.join(vox_path_list[:-1])
    return vox_path

def extract_vox_file(vox_grn_dir):
    vox_path_list = extract_vox_grn_key(vox_grn_dir).split('/')
    return vox_path_list[-1] if len(vox_path_list) > 1 else 'temp.mp3'


In [16]:
recovered_files['path'] = recovered_files['file'].apply(extract_vox_path)
recovered_files['filename'] = recovered_files['file'].apply(extract_vox_file)

## Files with bad filenames
It has been found that a number of files have bad filenames, including characters that cannot be used and marked as replacement character in pandas dataframes.

Lets recover those and see what the overlap is with our missing files.

In [11]:
import glob 
def check_for_glob_file(item_row):
    path_to_file = '/media/programs/' + item_row['Path'] + item_row['Filename'].replace('\ufffd', '*')
    files = glob.glob(path_to_file)
    return len(files) == 1    

file_descriptors['glob file exist'] = file_descriptors.apply(check_for_glob_file, axis=1)

In [17]:
file_descriptors['bad filename'] = file_descriptors['glob file exist'] & ~file_descriptors['file exists']
bad_fd = file_descriptors[file_descriptors['bad filename']].copy()
bad_fd['recovered'] = bad_fd['ID'].isin(recovered_files['ID'])
print(f'Bad filename in {len(bad_fd)} files and {sum(bad_fd["recovered"])} are in vox_grn.')

Bad filename in 557 files and 546 are in vox_grn.


Just 11 files? Are they the NaN ones?

In [30]:
bad_fd_unrecovered = bad_fd[bad_fd['recovered'] == False].copy()

Just check if all vox_grn files can be found.

In [20]:
def check_for_vox_grn_file(item_row):
    return os.path.isfile('/media/programs/' + item_row['path'] + '/' + item_row['filename'] )

recovered_files['exists'] = recovered_files.apply(check_for_vox_grn_file, axis=1)
print(f'Recovered files {sum(recovered_files["exists"])} out of {len(recovered_files)}')
    

Recovered files 7015 out of 7015


# Sanity Check
Make sure that the files downloaded match the items we think we have found.

In [31]:
# check that the found files have unique ids
print(f'There are {sum(recovered_files["ID"].duplicated())} duplicate IDs in the recovered files.')

There are 0 duplicate IDs in the recovered files.


In [33]:
# Now check that there is a file for each of the missing items.
found_files['found'] = found_files['ID'].isin(recovered_files["ID"])
print(f'Of the {len(found_files)} files we believed we actually found {sum(found_files["found"])} files.')
not_found = found_files[found_files['found'] == False]

Of the 6604 files we believed we actually found 6598 files.


# Merge
Now put each of the data sources into a state where they can be merged together.

What fields do I want? And what names?

grnvox_test had:
> "file", "audio", "iso", "program", "location", "item_no", "title", "item_start", "item_end", "seg_start", "seg_end", "seg"

I would like to add "item_type".


In [23]:
# check that the iso code matches for the matched items
merge_candidates = found_files[found_files['found']].copy()
merged_records = pd.merge(merge_candidates, recovered_files, on="ID", how='inner', validate='1:1')


In [24]:
print(merged_records.columns)

Index(['LanguageID', 'ISO', 'Language', 'Program', 'Track', 'Recordist',
       'Location', 'Year', 'Path', 'Filename', 'Size', 'Length', 'prog', 'ID',
       'file exists', 'found_prog', 'found', 'file', 'language name',
       'location', 'copyright', 'year', 'disguised', 'length', 'iso',
       'program', 'track', 'recovery_candidate', 'path', 'filename', 'exists'],
      dtype='object')


Sanity check the merge. Is the ISO the same? How many files are disguised? Is the length the same?

In [25]:
print(f'The ISO code is the same for {sum(merged_records["ISO"] == merged_records["iso"])} out of {len(merged_records)} records.')
print(f'The number of files with the voice disguised is {sum(merged_records["disguised"] == True)}')
print(f'The number of files where the length difference is signficant is {sum(abs(merged_records["Length"] - merged_records["length"]) > 1.0)}')

The ISO code is the same for 6552 out of 6598 records.
The number of files with the voice disguised is 20
The number of files where the length difference is signficant is 11


OK - Lets see why they are different.
First ISO records.

In [26]:
iso_different = merged_records[merged_records['ISO'] != merged_records['iso']]
mapping = {}
for rec in iso_different.itertuples():
    if rec.ISO in mapping:
        if rec.iso not in mapping[rec.ISO]:
            mapping[rec.ISO].append(rec.iso)
    else:
        mapping[rec.ISO] = list()
        mapping[rec.ISO].append(rec.iso)

for k, v in mapping.items():
    print(f'{k} mapped to {v}')

kvw mapped to ['aol']
wer mapped to ['kij']
sop mapped to ['yom']
ppo mapped to ['hmo']
lak mapped to ['ksp']
ksh mapped to ['bqv']
kxl mapped to ['kru']
spa mapped to ['tzo', 'mxb', 'xtn', 'xtm', 'mvg', 'mil', 'zaq', 'zar', 'zad', 'zac', 'zab', 'zpt', 'zps', 'ztg', 'zpe', 'ncl', 'nhi', 'tpx', 'tcf', 'maq', 'neq', 'mco', 'ott', 'otm', 'ote', 'ots', 'otq', 'toc']


But is the language the same?

In [27]:
print(f'The Language name is the same for {sum(merged_records["Language"] == merged_records["language name"])} out of {len(merged_records)} records.')
print(f'The Language name is the same for {sum(iso_different["Language"] == iso_different["language name"])} out of {len(iso_different)} records where the iso was different.')
print(f'The Location is the same for {sum(merged_records["Location"] == merged_records["location"])} out of {len(merged_records)} records.')
print(f'The Location is the same for {sum(iso_different["Location"] == iso_different["location"])} out of {len(iso_different)} records where the iso was different.')


The Language name is the same for 6404 out of 6598 records.
The Language name is the same for 6 out of 46 records where the iso was different.
The Location is the same for 5601 out of 6598 records.
The Location is the same for 12 out of 46 records where the iso was different.


I am going to say that the language given to grn-vox is more specific than the one in programs.

Now lets look at the length of the files.

In [28]:
# lets look at the records where the locations differ
location_diff = merged_records[merged_records["Location"] != merged_records["location"]]

The differences were only where the location was NaN or capitalisation was different or accents were on some letters.

In [29]:
merged_records['length_diff'] = merged_records.Length - merged_records.length


OK - so none of the lengths are zero and only a couple differ significantly - take the vox-grn data for length.

Now set things up so that the file descriptors of the files we previously used are ready to have the newly found files appended.

In [35]:
original_descriptors = file_descriptors[file_descriptors['file exists']].copy()
original_descriptors.drop(columns=['LanguageID', 'Recordist', 'Size', 'file exists', 'Program', 'glob file exist', 'bad filename'], inplace=True)
rename_map = { 'ISO' : 'iso', 'Language' : 'language_name', 'prog' : 'program', 'Year' : 'year',
               'Track' : 'track', 'Location' : 'location', 'Path' : 'path', 'Filename' : 'filename', 'Length' : 'length' }
original_descriptors.rename(columns=rename_map, inplace=True)


In [36]:
print(merged_records.columns)

Index(['LanguageID', 'ISO', 'Language', 'Program', 'Track', 'Recordist',
       'Location', 'Year', 'Path', 'Filename', 'Size', 'Length', 'prog', 'ID',
       'file exists', 'found_prog', 'found', 'file', 'language name',
       'location', 'copyright', 'year', 'disguised', 'length', 'iso',
       'program', 'track', 'recovery_candidate', 'path', 'filename', 'exists',
       'length_diff'],
      dtype='object')


In [39]:
new_descriptors = merged_records.drop(columns=['LanguageID', 'ISO', 'Language', 
     'Program', 'Track', 'Recordist', 'Location', 'Year', 'Path', 'Filename', 'Size', 'Length', 
     'prog', 'file exists', 'found_prog', 'found', 'file', 'copyright', 'disguised',
     'recovery_candidate', 'length_diff', 'exists'])
new_descriptors.rename(columns={ 'language name' : 'language_name' }, inplace=True)


In [42]:
bad_fname = bad_fd_unrecovered.drop(columns=['LanguageID', 'Recordist', 'Size', 'file exists', 'Program', 'glob file exist', 'bad filename', 'recovered'])
rename_map = { 'ISO' : 'iso', 'Language' : 'language_name', 'prog' : 'program', 'Year' : 'year',
               'Track' : 'track', 'Location' : 'location', 'Path' : 'path', 'Filename' : 'filename', 'Length' : 'length' }
bad_fname.rename(columns=rename_map, inplace=True)


In [43]:
print(original_descriptors.columns)
print(new_descriptors.columns)
print(bad_fname.columns)

Index(['iso', 'language_name', 'track', 'location', 'year', 'path', 'filename',
       'length', 'program', 'ID'],
      dtype='object')
Index(['ID', 'language_name', 'location', 'year', 'length', 'iso', 'program',
       'track', 'path', 'filename'],
      dtype='object')
Index(['iso', 'language_name', 'track', 'location', 'year', 'path', 'filename',
       'length', 'program', 'ID'],
      dtype='object')


In [44]:
fd = pd.concat([original_descriptors, new_descriptors, bad_fname], ignore_index=True)

# Save the file
Now that we have formed a file we can save it away.

In [45]:
fd.to_csv("../../data/records_with_voxgrn_files.csv")
print(f'The columns of items with records are:\n{fd.columns}')


The columns of items with records are:
Index(['iso', 'language_name', 'track', 'location', 'year', 'path', 'filename',
       'length', 'program', 'ID'],
      dtype='object')


# Generate program items with files

In [28]:
print(items.columns)

Index(['Program Number', 'Program Item Number', 'Tape Side', 'Track Number',
       'Original Recording Number', 'Original Item Number', 'Title',
       'Vernacular Item Title', 'Language Number', 'Language Name',
       'Item Start Time', 'Item Time', 'Finish Time', 'Original Time',
       'Script Number', 'Script Name', 'Picture Number', 'Item Type',
       'Comments', 'Entered By', 'Enter On Date', 'program', 'ID'],
      dtype='object')


In [46]:
def usable_types(item_row):
    unusable_items = ['Instrumental', 'Sound Effect', 'Announcement', 'Bridge']
    return item_row['Item Type'] not in unusable_items

items['usable'] = items.apply(usable_types, axis=1)
usable_items = items[items["usable"]].copy()

usable_items.drop(['usable'], inplace=True, axis=1)
print(f'There are {usable_items.shape[0]} usable items out of {items.shape[0]} total items.')

There are 261457 usable items out of 267681 total items.


In [47]:
items_with_records = pd.merge(fd, usable_items, on="ID", how='inner', validate='1:m')
print(f'Of {usable_items.shape[0]} records {items_with_records.shape[0]} were associated with records')

Of 261457 records 248682 were associated with records


In [48]:
print(f'The columns of items with records are:\n{items_with_records.columns}')

The columns of items with records are:
Index(['iso', 'language_name', 'track', 'location', 'year', 'path', 'filename',
       'length', 'program_x', 'ID', 'Program Number', 'Program Item Number',
       'Tape Side', 'Track Number', 'Original Recording Number',
       'Original Item Number', 'Title', 'Vernacular Item Title',
       'Language Number', 'Language Name', 'Item Start Time', 'Item Time',
       'Finish Time', 'Original Time', 'Script Number', 'Script Name',
       'Picture Number', 'Item Type', 'Comments', 'Entered By',
       'Enter On Date', 'program_y'],
      dtype='object')


In [49]:
# drop the columns that we do not want
items_with_records.drop(columns=['Program Number', 'Tape Side', 'Track Number', 'Original Recording Number', 'Original Item Number', 
                                 'Vernacular Item Title', 'Language Number', 'Script Number', 'Script Name', 'Picture Number', 'Entered By', 
                                 'Enter On Date'], inplace=True)

Some more consistency checks

In [50]:
print(f'The program differs in {sum(items_with_records.program_x != items_with_records.program_y)} records.')

The program differs in 0 records.


In [51]:
items_with_records.drop(columns=['program_x', 'Comments', 'Language Name'], inplace=True)

In [52]:
print(f'The columns of items with records are:\n{items_with_records.columns}')

The columns of items with records are:
Index(['iso', 'language_name', 'track', 'location', 'year', 'path', 'filename',
       'length', 'ID', 'Program Item Number', 'Title', 'Item Start Time',
       'Item Time', 'Finish Time', 'Original Time', 'Item Type', 'program_y'],
      dtype='object')


In [53]:
items_with_records.rename(columns={'Program Item Number': 'item', 'Title': 'title', 'Item Start Time': 'start',
        'Item Time': 'duration', 'Finish Time': 'end', 'Item Type': 'type', 'program_y' : 'program'}, inplace=True)
items_with_records.drop(columns=['Original Time'], inplace=True)

# Save File

In [54]:
items_with_records.to_csv("../../data/items_with_records_with_voxgrn_files.csv")
print(f'The columns of items with records are:\n{items_with_records.columns}')

The columns of items with records are:
Index(['iso', 'language_name', 'track', 'location', 'year', 'path', 'filename',
       'length', 'ID', 'item', 'title', 'start', 'duration', 'end', 'type',
       'program'],
      dtype='object')
